In [1]:
from data_import import import_user_ratings, import_semantic, generate_random_user_ratings_input
from bliga import BookRecommendationSystem
from tuner import HyperparameterTuner
from evaluation import Evaluator


# File paths for data import
user_ratings_file = "user_ratings.csv"
semantic_file = "semantic.csv"

# Import data
user_ratings = import_user_ratings(user_ratings_file)
semantic = import_semantic(semantic_file)

# Generate user_ratings_input
user_ratings_input = generate_random_user_ratings_input(200)

In [2]:
import itertools

# Given lists for each parameter
pop_size = [20, 50, 100]
mutation_rate = [0.1]
no_generations = [10,20]
crossover = ["one_point", "uniform"]
mutation = ["inversion"]
no_rec = [5, 15]

# Generate all possible combinations
combinations_list = list(itertools.product(pop_size, mutation_rate, no_generations, crossover, mutation, no_rec))

# Print the generated combinations
for combo in combinations_list:
    print(combo)

(20, 0.1, 10, 'one_point', 'inversion', 5)
(20, 0.1, 10, 'one_point', 'inversion', 15)
(20, 0.1, 10, 'uniform', 'inversion', 5)
(20, 0.1, 10, 'uniform', 'inversion', 15)
(20, 0.1, 20, 'one_point', 'inversion', 5)
(20, 0.1, 20, 'one_point', 'inversion', 15)
(20, 0.1, 20, 'uniform', 'inversion', 5)
(20, 0.1, 20, 'uniform', 'inversion', 15)
(50, 0.1, 10, 'one_point', 'inversion', 5)
(50, 0.1, 10, 'one_point', 'inversion', 15)
(50, 0.1, 10, 'uniform', 'inversion', 5)
(50, 0.1, 10, 'uniform', 'inversion', 15)
(50, 0.1, 20, 'one_point', 'inversion', 5)
(50, 0.1, 20, 'one_point', 'inversion', 15)
(50, 0.1, 20, 'uniform', 'inversion', 5)
(50, 0.1, 20, 'uniform', 'inversion', 15)
(100, 0.1, 10, 'one_point', 'inversion', 5)
(100, 0.1, 10, 'one_point', 'inversion', 15)
(100, 0.1, 10, 'uniform', 'inversion', 5)
(100, 0.1, 10, 'uniform', 'inversion', 15)
(100, 0.1, 20, 'one_point', 'inversion', 5)
(100, 0.1, 20, 'one_point', 'inversion', 15)
(100, 0.1, 20, 'uniform', 'inversion', 5)
(100, 0.1, 20, 

In [5]:
pop_size, mutation_rate, num_generations, crossover_func, mutation_func, no_rec = 20, 0.1, 10, 'one_point', 'inversion', 5


# Using BookRecommendationSystem
recommender = BookRecommendationSystem(user_ratings_input, user_ratings, semantic)

# Call the genetic_algorithm method with appropriate arguments
best_solution, semratings_dict, fitness_scores_dict, best_solutions_by_generation = recommender.genetic_algorithm(
pop_size, mutation_rate, num_generations, crossover_func, mutation_func, no_rec
)


In [13]:
best_solutions_by_generation

{0: [26, 99, 6, 70, 35],
 1: [26, 99, 6, 35, 70],
 2: [26, 99, 6, 70, 35],
 3: [26, 99, 6, 70, 35],
 4: [26, 99, 6, 70, 35],
 5: [26, 99, 6, 70, 35],
 6: [26, 99, 6, 70, 35],
 7: [26, 99, 6, 70, 35],
 8: [26, 99, 6, 70, 35],
 9: [26, 99, 6, 70, 35]}

In [6]:
best_solution

(118, 192, 99, 35, 180)

In [8]:
combinations_list[1]

(20, 0.1, 10, 'one_point', 'inversion', 15)

In [3]:
import itertools
import pandas as pd
import os
from datetime import datetime
from tqdm import tqdm  # Import the tqdm function

# Given lists for each parameter
pop_size = [20, 50, 100]
mutation_rate = [0.1]
no_generations = [5, 15]
crossover = ["one_point", "uniform"]
mutation = ["inversion", "swap"]
no_rec = [5, 15]

# Generate all possible combinations
combinations_list = list(itertools.product(pop_size, mutation_rate, no_generations, crossover, mutation, no_rec))

# Define your genetic algorithm function here (genetic_algorithm_function)
current_dir = os.getcwd()

filename = 'results.xlsx'
filename = os.path.join(current_dir, filename)

results_list = []  # Initialize the results_list before the loop
checkpoint_interval = 5  # Set the checkpoint interval in iterations

try:
    # Use tqdm to create a progress bar
    for i, (pop_size, mutation_rate, num_generations, crossover_func, mutation_func, no_rec) in enumerate(tqdm(combinations_list)):
        try:
            start_time = datetime.now()

            # Using BookRecommendationSystem
            recommender = BookRecommendationSystem(user_ratings_input, user_ratings, semantic)

            # Call the genetic_algorithm method with appropriate arguments
            best_solution, semratings_dict, fitness_scores_dict, best_solutions_by_generation = recommender.genetic_algorithm(
                pop_size, mutation_rate, num_generations, crossover_func, mutation_func, no_rec
            )

            end_time = datetime.now()

            elapsed_time = end_time - start_time

            elapsed_time = elapsed_time.total_seconds()

            best_solutions_by_generations_fitness = []

            for generation, solution in best_solutions_by_generation.items():
                solution = tuple(solution)
                if solution in fitness_scores_dict:
                    fitness_info = fitness_scores_dict[solution]['total_predicted_rating']
                    best_solutions_by_generations_fitness.append(fitness_info)

            best_solution_fitness = fitness_scores_dict[best_solution]['total_predicted_rating']

            # Append the results to the list
            results_list.append((pop_size, mutation_rate, num_generations, crossover_func, mutation_func, no_rec, best_solution, best_solution_fitness, best_solutions_by_generations_fitness, best_solutions_by_generation, elapsed_time))

            # Periodically save the results to a file
            if i % checkpoint_interval == 0:
                results_df = pd.DataFrame(results_list, columns=['Population Size', 'Mutation Rate', 'Number of Generations', 'Crossover Function', 'Mutation Function', 'No. of Recommendations', 'Best Solution', 'Best Solution Fitness', 'Best Solutions By Generations Fitness', 'Best Solution By Generation', 'Elapsed Time'])
                # Save the DataFrame to the unique filename with a checkpoint number
                checkpoint_filename = f"checkpoint_{i}.xlsx"
                checkpoint_filename = os.path.join(current_dir, checkpoint_filename)
                results_df.to_excel(checkpoint_filename, index=False)
                print(f"Checkpoint {i} saved successfully.")

        except Exception as e:
            # Handle specific exceptions that occur during execution
            print(f"Error occurred for combination: {combo}. Error message: {str(e)}")

finally:
    # Save the final results to a file
    results_df = pd.DataFrame(results_list, columns=['Population Size', 'Mutation Rate', 'Number of Generations', 'Crossover Function', 'Mutation Function', 'No. of Recommendations', 'Best Solution', 'Best Solution Fitness', 'Best Solutions By Generations Fitness', 'Best Solution By Generation','Elapsed Time'])
    results_df.to_excel(filename, index=False)
    print("Results saved successfully.")

  2%|▏         | 1/48 [00:22<17:35, 22.45s/it]

Checkpoint 0 saved successfully.


 12%|█▎        | 6/48 [02:03<12:15, 17.50s/it]

Checkpoint 5 saved successfully.


 23%|██▎       | 11/48 [03:26<11:17, 18.31s/it]

...

Checkpoint 40 saved successfully.


 96%|█████████▌| 46/48 [52:32<06:24, 192.13s/it]

Checkpoint 45 saved successfully.


100%|██████████| 48/48 [1:03:35<00:00, 79.49s/it] 


Results saved successfully.
